In [1]:
import pandas as pd

X_train = pd.read_csv("../data/processed/X_train.csv")
y_train = pd.read_csv("../data/processed/y_train.csv")["item_cnt_month"]

X_valid = pd.read_csv("../data/processed/X_valid.csv")
y_valid = pd.read_csv("../data/processed/y_valid.csv")["item_cnt_month"]

X_test  = pd.read_csv("../data/processed/X_test.csv")


In [2]:
#Regresión ridge 
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import numpy as np

model_ridge = Ridge(alpha=1.0)

model_ridge.fit(X_train, y_train)

y_valid_pred = model_ridge.predict(X_valid)

rmse = np.sqrt(mean_squared_error(y_valid, y_valid_pred))
rmse


np.float64(0.9806157819368521)

In [3]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

gbr = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=5,
    random_state=42
)

gbr.fit(X_train, y_train)

y_valid_pred = gbr.predict(X_valid)

rmse = np.sqrt(mean_squared_error(y_valid, y_valid_pred))
rmse


np.float64(0.9563481035866696)

In [ ]:
#El GBR tiene mejor RMSE que ridge, por eso optamos por este sobre el baseline 0.9806 vs 0.9563

In [ ]:
#La mejora sugiere que existen relaciones no lineales entre ventas y los features temporales (lags y agregados).La validación se hizo usando un split temporal (mes 33)
#

In [ ]:
import numpy as np
import pandas as pd
import joblib

# El mejor modelo fue gbr
best_model = gbr

# Re-entrenar con todo (train + valid)
X_all = pd.concat([X_train, X_valid], ignore_index=True)
y_all = pd.concat([y_train, y_valid], ignore_index=True)

best_model.fit(X_all, y_all)

# Predecir test
test_pred = best_model.predict(X_test)

#Clipping final
test_pred = np.clip(test_pred, 0, 20)



In [8]:
test_raw = pd.read_csv("C:/Users/emil_/Desktop/ITAM_Maestrtía/GranEscala/Tarea01/competitive-data-science-predict-future-sales/test.csv")
submission = pd.DataFrame({
    "ID": test_raw["ID"],
    "item_cnt_month": test_pred
})

submission.to_csv("submission.csv", index=False)
submission.head()



,ID,item_cnt_month
0,0,0.530103
1,1,0.079248
2,2,1.044704
3,3,0.265228
4,4,0.746594


In [9]:
joblib.dump(best_model, "modelo_final.joblib")

['modelo_final.joblib']

In [10]:
import os
os.getcwd()


'C:\\Users\\emil_\\downloads\\Emi-y-Mora'